# S3 Access Logs report

# Setup

In [ ]:
!pip install --upgrade pandas

## Parameters and AWS session configuration

In [ ]:
import boto3
import os
import io
import time
import pandas as pd

pd.options.display.width = 0

%env S3_ACCESS_LOGS_BUCKET=<YOUR_S3_ACCESS_LOGS_BUCKET>
%env OUTPUT_BUCKET_NAME=<YOUR_OUTPUT_BUCKET>
%env ATHENA_DATABASE_NAME=s3_access_logs
%env ATHENA_TABLE_NAME=s3_access_logs_table
%env ATHENA_VIEW_NAME=s3_access_logs_view


# Access Logs analysis

## Athena configuration

In [ ]:
s3_input = 's3://{}/'.format(os.environ['S3_ACCESS_LOGS_BUCKET'])
s3_output_bucket_name = process.env['OUTPUT_BUCKET_NAME']
s3_output_path = 's3://{}'.format(s3_output_bucket_name)
database = process.env['ATHENA_DATABASE_NAME']
table = process.env['ATHENA_TABLE_NAME']
view_name = process.env['ATHENA_VIEW_NAME']

In [ ]:
client = boto3.client('athena')

# Helper function for executing athena queries
def run_query(query, s3_output, database=None):
    if database is None:
        response = client.start_query_execution(QueryString=query, ResultConfiguration={'OutputLocation': s3_output})
    else:
        response = client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={'Database': database},
            ResultConfiguration={'OutputLocation': s3_output}
        )
    return response

def obtain_data(filename):
    try:
        objectKey = filename + '.csv'
        resource = boto3.resource('s3')
        response = resource.Bucket(s3_output_bucket_name).Object(key= objectKey).get()
        return pd.read_csv(io.BytesIO(response['Body'].read()), encoding='utf8')   
    except Exception as e:
        print(e)

## Create Athena database, table, and view

In [ ]:
# SQL commands to create Athena S3 Access Logs database, table definition and view
create_database = "CREATE DATABASE IF NOT EXISTS %s;" % (database)

create_table = \
    """
CREATE EXTERNAL TABLE `%s`(
  `bucketowner` string COMMENT '',
  `bucket` string COMMENT '',
  `requestdatetime` string COMMENT '',
  `remoteip` string COMMENT '',
  `requester` string COMMENT '',
  `requestid` string COMMENT '',
  `operation` string COMMENT '',
  `key` string COMMENT '',
  `requesturi_operation` string COMMENT '',
  `requesturi_key` string COMMENT '',
  `requesturi_httpprotoversion` string COMMENT '',
  `httpstatus` string COMMENT '',
  `errorcode` string COMMENT '',
  `bytessent` bigint COMMENT '',
  `objectsize` bigint COMMENT '',
  `totaltime` string COMMENT '',
  `turnaroundtime` string COMMENT '',
  `referrer` string COMMENT '',
  `useragent` string COMMENT '',
  `versionid` string COMMENT '',
  `hostid` string COMMENT '',
  `sigv` string COMMENT '',
  `ciphersuite` string COMMENT '',
  `authtype` string COMMENT '',
  `endpoint` string COMMENT '',
  `tlsversion` string COMMENT '')
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
  'input.regex'='([^ ]*) ([^ ]*) \\\[(.*?)\\\] ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) \\\\\\\"([^ ]*) ([^ ]*) (- |[^ ]*)\\\\\\\" (-|[0-9]*) ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) (\\\"[^\\\"]*\\\") ([^ ]*)(?: ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*) ([^ ]*))?.*$')
STORED AS INPUTFORMAT
  'org.apache.hadoop.mapred.TextInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  '%s'
TBLPROPERTIES (
  'transient_lastDdlTime'='1573646113')
    """ % ( table, s3_input )

create_view = \
 """
 CREATE OR REPLACE VIEW %s AS 
 SELECT DISTINCT bucket, tlsversion, requester
 FROM %s""" % (view_name, table)

# creating the database if not exists
create_db_result = run_query(create_database, s3_output_path)

# Create the flowlogs table combining all collected data from the bucket
create_s3_access_logs_table = run_query(create_table, s3_output_path, database)

time.sleep(2) # So the view will be created after the table is created

#Create 
create_s3_view = run_query(create_view, s3_output_path, database)
# print(create_port_address_view)

## Query the view with AWS Athena and obtain the results from S3 bucket

In [ ]:
response = run_query("SELECT DISTINCT * FROM {}.{} WHERE tlsversion NOT IN ('TLSv1.0', 'TLSv1.1', 'TLSv1.2', 'TLSv1.3')".format(database, view_name), s3_output_path, database)
execution_id = response['QueryExecutionId']
athena = boto3.client('athena')

status = "RUNNING"
status = athena.get_query_execution(QueryExecutionId=execution_id)['QueryExecution']['Status']['State']
while status == "RUNNING":
    print('Query still running')
    time.sleep(3)
    status = athena.get_query_execution(QueryExecutionId=execution_id)['QueryExecution']['Status']['State']
print('Query is DONE!');


# Results
The following buckets, if any, have had unsecure connections. For each such case, you may find the requester which made the unsecure call.

In [ ]:
results = obtain_data(execution_id)
if len(results) == 0:
    print('No unsecure connections detected')
else:
    print(results)
